Implementing MLP

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import USPS


In [2]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(16*16, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 10)
        )

    def forward(self, x):
        x = x.view(x.size(0), -1)
        return self.layers(x)

In [4]:
import h5py
def load_usps_dataset(h5_file):
    with h5py.File(h5_file, 'r') as hf:
        train = hf.get('train')
        X_train = train.get('data')[:]
        y_train = train.get('target')[:]
        test = hf.get('test')
        X_test = test.get('data')[:]
        y_test = test.get('target')[:]
    return X_train, y_train, X_test, y_test

# Load the dataset
X_train, y_train, X_test, y_test = load_usps_dataset('usps.h5')

In [16]:
from torch.utils.data import TensorDataset
# Convert the data to PyTorch tensors
X_train = torch.tensor(X_train)
X_test = torch.tensor(X_test)

# Convert the target data to type torch.long
y_train = torch.tensor(y_train, dtype=torch.long)
y_test = torch.tensor(y_test, dtype=torch.long)

# Create tensor datasets
train_ds = TensorDataset(X_train, y_train)
test_ds = TensorDataset(X_test, y_test)

# Create data loaders
train_loader = DataLoader(train_ds, batch_size=64, shuffle=True)
test_loader = DataLoader(test_ds, batch_size=1000)

<ipython-input-16-51fa0b267191>:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train = torch.tensor(X_train)
<ipython-input-16-51fa0b267191>:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test = torch.tensor(X_test)
<ipython-input-16-51fa0b267191>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.tensor(y_train, dtype=torch.long)
<ipython-input-16-51fa0b267191>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True)

In [17]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.fc_layers = nn.Sequential(
            nn.Linear(64 * 4 * 4, 128),
            nn.ReLU(),
            nn.Linear(128, 10)
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(x.size(0), -1)
        return self.fc_layers(x)

In [37]:
def train(model, device, train_loader, optimizer, criterion):
    model.train()
    total_loss = 0
    for data, target in train_loader:
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    average_loss = total_loss / len(train_loader)
    return average_loss

In [38]:
def evaluate(model, device, data_loader):
    model.eval()
    correct = 0
    with torch.no_grad():
        for data, target in data_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    return correct / len(data_loader.dataset)

In [39]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# For MLP
mlp_model = MLP().to(device)
optimizer = optim.Adam(mlp_model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()



In [40]:
num_epochs = 10
for epoch in range(num_epochs):
    train(mlp_model, device, train_loader, optimizer, criterion)
mlp_accuracy = evaluate(mlp_model, device, test_loader)

In [41]:
# For CNN
cnn_model = CNN().to(device)
optimizer = optim.Adam(cnn_model.parameters(), lr=0.001)

In [42]:
X_train_cnn = X_train.view(-1, 1, 16, 16)
X_test_cnn = X_test.view(-1, 1, 16, 16)

# Tensor datasets for CNN
train_ds_cnn = TensorDataset(X_train_cnn, y_train)
test_ds_cnn = TensorDataset(X_test_cnn, y_test)

# Data loaders for CNN
train_loader_cnn = DataLoader(train_ds_cnn, batch_size=64, shuffle=True)
test_loader_cnn = DataLoader(test_ds_cnn, batch_size=1000)

In [43]:
for epoch in range(num_epochs):
    train(cnn_model, device, train_loader_cnn, optimizer, criterion)
cnn_accuracy = evaluate(cnn_model, device, test_loader_cnn)

Compare the models in terms of accuracy, recall, precision, and confusion matrix.

In [44]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, confusion_matrix
def get_predictions_and_targets(model, data_loader, device):
    model.eval()
    predictions = []
    targets = []

    with torch.no_grad():
        for data, target in data_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            preds = output.argmax(dim=1)
            predictions.extend(preds.tolist())
            targets.extend(target.tolist())

    return predictions, targets




In [45]:
# Get predictions and targets for both models
mlp_preds, mlp_targets = get_predictions_and_targets(mlp_model, test_loader, device)
cnn_preds, cnn_targets = get_predictions_and_targets(cnn_model, test_loader_cnn, device)


In [46]:
# Calculate metrics for MLP
mlp_accuracy = accuracy_score(mlp_targets, mlp_preds)
mlp_recall = recall_score(mlp_targets, mlp_preds, average='macro')
mlp_precision = precision_score(mlp_targets, mlp_preds, average='macro')
mlp_conf_matrix = confusion_matrix(mlp_targets, mlp_preds)

In [47]:
# Calculate metrics for CNN
cnn_accuracy = accuracy_score(cnn_targets, cnn_preds)
cnn_recall = recall_score(cnn_targets, cnn_preds, average='macro')
cnn_precision = precision_score(cnn_targets, cnn_preds, average='macro')
cnn_conf_matrix = confusion_matrix(cnn_targets, cnn_preds)

Use the Pytorch TensorBoard toolkit to show the precision-recall curve and loss function.

In [48]:
from torch.utils.tensorboard import SummaryWriter
writer_mlp = SummaryWriter('runs/mlp_experiment')
writer_cnn = SummaryWriter('runs/cnn_experiment')

In [51]:
# MLP Model Training with TensorBoard Logging
for epoch in range(num_epochs):
    train(mlp_model, device, train_loader, optimizer, criterion)
    loss_value =train(mlp_model, device, train_loader, optimizer, criterion)
    writer_mlp.add_scalar('Loss/train', loss_value, epoch)
    # You might want to log other metrics as well

# CNN Model Training with TensorBoard Logging
for epoch in range(num_epochs):
    train(cnn_model, device, train_loader_cnn, optimizer, criterion)
    loss_value = train(cnn_model, device, train_loader_cnn, optimizer, criterion)
    writer_cnn.add_scalar('Loss/train', loss_value, epoch)
    # You might want to log other metrics as well

# Close the writers after training
writer_mlp.close()
writer_cnn.close()